# Import Libraries

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import gzip
import json
import re
import os
from tqdm import tqdm
import requests
from io import BytesIO
from pickle import dump, load
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn import preprocessing
from sklearn.metrics import pairwise_distances
import scipy.sparse
from scipy.sparse import hstack
from scipy import stats
from scipy.sparse import coo_matrix
from surprise import Reader, Dataset
from surprise import SVD, model_selection, accuracy
from PIL import Image
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model


# Load Data

In [92]:
df_ratings = pd.read_parquet("C:/Users/adria/Documents/Projects/Books/Resources/DataFrames/SentimentAnalysis/200plusRatingsPerUser_60plusRatingsPerBook/results_df.parquet")

In [93]:
df_ratings.rename(columns={'User_id':'user_id'}, inplace=True)

In [97]:
df_ratings[df_ratings['title'].str.contains('Harry', case=False, na=False)]

,Id,roberta_neg,roberta_neu,roberta_pos,book_id,title,user_id,profileName,review_score,review_text


In [100]:
df_ratings.columns

Index(['Id', 'roberta_neg', 'roberta_neu', 'roberta_pos', 'book_id', 'title',
       'user_id', 'profileName', 'review_score', 'review_text'],
      dtype='object')

In [106]:
df_books = pd.read_parquet('C:/Users/adria/Documents/Projects/Books/Resources/DataFrames/filtered_books_data115.parquet')

In [98]:
df_books.head(0)

,book_id,title,description,authors,image,previewLink,publisher,publishedDate,infoLink,genre


In [105]:
desired_columns = ['Id', 'roberta_neg', 'roberta_neu', 'roberta_pos', 'title',
       'user_id', 'profileName', 'review_score', 'review_text']
df_ratings_copy = df_ratings[desired_columns].copy()
df_ratings_copy.head(0)

,Id,roberta_neg,roberta_neu,roberta_pos,title,user_id,profileName,review_score,review_text


In [110]:
df_ratings_copy = pd.merge(df_ratings_copy, df_books[['title', 'book_id']], on='title', how='inner')
df_ratings_copy

,Id,roberta_neg,roberta_neu,roberta_pos,title,user_id,profileName,review_score,review_text,book_id
0,1747,0.004265,0.053986,0.941749,The Scarlet Letter A Romance,AUM3YMZ0YRJE0,Robert J. Crawford,5.0,"WHen I finally started reading this classic, I...",B000N6DDJQ
1,1807,0.011064,0.125440,0.863495,The Scarlet Letter A Romance,AMKZHBOK7VMQR,R. J. Marsella,5.0,Hawthorne wrote a masterful work of historical...,B000N6DDJQ
2,1820,0.118758,0.562613,0.318629,The Scarlet Letter A Romance,AWLFVCT9128JV,"Dave_42 ""Dave_42""",4.0,I don't suppose anything can ruin a book more ...,B000N6DDJQ
3,1829,0.542925,0.406122,0.050953,The Scarlet Letter A Romance,ABN5K7K1TM1QA,Dennis Littrell,5.0,"""All have sinned and fall short of the glory o...",B000N6DDJQ
4,1830,0.360523,0.539366,0.100111,The Scarlet Letter A Romance,A3IKBHODOTYYHM,"fra7299 ""fra7299""",4.0,I think the many readers who were forced to pa...,B000N6DDJQ
...,...,...,...,...,...,...,...,...,...,...
13837,2978030,0.163704,0.662723,0.173573,The Hobbit or There and Back Again,A21W6PCDZWG15R,J C E Hitchcock,4.0,This book tells of the hobbit Bilbo Baggins an...,B000Q032UY
13838,2978032,0.006546,0.074743,0.918711,The Hobbit or There and Back Again,A2C27IQUH9N1Z,audrey,5.0,This review discusses the graphic novel of Tol...,B000Q032UY
13839,2978035,0.118400,0.510296,0.371304,The Hobbit or There and Back Again,A1D2C0WDCSHUWZ,"E. A Solinas ""ea_solinas""",5.0,The face of literature changed forever with th...,B000Q032UY
13840,2978087,0.002854,0.016767,0.980378,The Hobbit or There and Back Again,AFYYHRPSFBLWS,Ash Ryan,4.0,"Very well written, which makes the story seem ...",B000Q032UY


In [111]:
df_ratings_copy.book_id.nunique()

115

# Pre-Processing Data

*Process Publishing Date that lacks format*

In [142]:
df_books_copy = df_books.copy()

df_books_copy.publishedDate = df_books_copy.publishedDate.apply(lambda x: x + '-01-01' if isinstance(x, str) and len(x) == 4 else x)
df_books_copy.publishedDate = df_books_copy.publishedDate.apply(lambda x: x + '-01' if isinstance(x, str) and len(x) == 7 else x)


df_books_copy.publishedDate = pd.to_datetime(df_books_copy.publishedDate, errors='coerce')
df_books_copy['publication_year'] = df_books_copy['publishedDate'].dt.year

median_publication_year = df_books_copy['publication_year'].median()
df_books_copy['publication_year'] = df_books_copy['publication_year'].fillna(median_publication_year).astype(int)

df_books_copy = df_books_copy.drop('publishedDate', axis=1)

df_books_copy.info(0)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115 entries, 0 to 114
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           115 non-null    object
 1   title             115 non-null    object
 2   description       99 non-null     object
 3   authors           105 non-null    object
 4   image             99 non-null     object
 5   previewLink       107 non-null    object
 6   publisher         91 non-null     object
 7   infoLink          107 non-null    object
 8   genre             89 non-null     object
 9   publication_year  115 non-null    int32 
dtypes: int32(1), object(9)
memory usage: 8.7+ KB


In [144]:
df_books_copy.head(0)

,book_id,title,description,authors,image,previewLink,publisher,infoLink,genre,publication_year


In [226]:
df_ratings_copy.info(0)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13842 entries, 0 to 13841
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Id            13842 non-null  int64  
 1   roberta_neg   13842 non-null  float32
 2   roberta_neu   13842 non-null  float32
 3   roberta_pos   13842 non-null  float32
 4   book_id       13842 non-null  object 
 5   title         13842 non-null  object 
 6   user_id       13842 non-null  object 
 7   profileName   13842 non-null  object 
 8   review_score  13842 non-null  float64
 9   review_text   13842 non-null  object 
dtypes: float32(3), float64(1), int64(1), object(5)
memory usage: 919.3+ KB


In [227]:
# Function to process Data

def preprocess_numeric_data(dataframe, df_name):
  if df_name == 'df_books_copy': # Process Data only for books DataFrame
    # Only publishedDate needs to be processed as numeric. 
    dataframe.publishedDate = dataframe.publishedDate.apply(lambda x: x + '-01-01' if isinstance(x, str) and len(x) == 4 else x)
    dataframe.publishedDate = dataframe.publishedDate.apply(lambda x: x + '-01' if isinstance(x, str) and len(x) == 7 else x)
    dataframe.publishedDate = pd.to_datetime(dataframe.publishedDate, errors='coerce')
    dataframe['publication_year'] = dataframe['publishedDate'].dt.year
    median_publication_year = dataframe['publication_year'].median()
    dataframe['publication_year'] = dataframe['publication_year'].fillna(median_publication_year).astype(int)
    dataframe = dataframe.drop('publishedDate', axis=1)

  elif df_name == 'df_ratings_copy': # Process Data only for users Data Base df_ratings_copy
    # Only review_score to be processed from float64 to int32. 
    dataframe['review_score'] = dataframe['review_score'].astype(int)

  return dataframe

def preprocess_text_data(dataframe, df_name):
  dataframe["book_id"] = dataframe["book_id"].astype(str)
  if df_name == 'df_books_numeric_mod':
    # dataframe["title"] = dataframe["title"].str.replace("[^a-zA-Z0-9 ]", "", regex=True)
    # dataframe["title"] = dataframe["title"].str.lower()
    # dataframe["title"] = dataframe["title"].str.replace("\s+", " ", regex=True)
    # dataframe = dataframe[dataframe["title"].str.len() > 0]
    
    dataframe["description"] = dataframe["description"].str.replace("[^a-zA-Z0-9 ]", "", regex=True)
    dataframe["description"] = dataframe["description"].str.lower()
    dataframe["description"] = dataframe["description"].str.replace("\s+", " ", regex=True)
    dataframe['description'] = dataframe['description'].fillna('no description')


    dataframe["publisher"] = dataframe["publisher"].str.replace("[^a-zA-Z0-9 ]", "", regex=True)
    dataframe["publisher"] = dataframe["publisher"].str.lower()
    dataframe["publisher"] = dataframe["publisher"].str.replace("\s+", " ", regex=True)

    # Handling nulls
    dataframe['authors'] = dataframe['authors'].fillna('other')
    dataframe['publisher'] = dataframe['publisher'].fillna('other')
    dataframe['genre'] = dataframe['genre'].fillna('other')
    dataframe['image'] = dataframe['image'].fillna('https://www.google.com/url?sa=i&url=https%3A%2F%2Fislandpress.org%2Fbooks%2Fwild-cats&psig=AOvVaw1KOBgRGHa-CM0hMZUhZ968&ust=1700839051074000&source=images&cd=vfe&opi=89978449&ved=0CBEQjRxqFwoTCMCaz6212oIDFQAAAAAdAAAAABAJ')
    dataframe['previewLink'] = dataframe['previewLink'].fillna('No Link')
    dataframe['infoLink'] = dataframe['infoLink'].fillna('No Link')

  elif df_name == 'df_ratings_numeric_mod':
    # dataframe["title"] = dataframe["title"].str.replace("[^a-zA-Z0-9 ]", "", regex=True)
    # dataframe["title"] = dataframe["title"].str.lower()
    # dataframe["title"] = dataframe["title"].str.replace("\s+", " ", regex=True)
    
    dataframe["profileName"] = dataframe["profileName"].str.replace("[^a-zA-Z0-9 ]", "", regex=True)
    dataframe["profileName"] = dataframe["profileName"].str.lower()
    dataframe["profileName"] = dataframe["profileName"].str.replace("\s+", " ", regex=True)

    dataframe["review_text"] = dataframe["review_text"].str.replace("[^a-zA-Z0-9 ]", "", regex=True)
    dataframe["review_text"] = dataframe["review_text"].str.lower()
    dataframe["review_text"] = dataframe["review_text"].str.replace("\s+", " ", regex=True)

    dataframe["user_id"] = dataframe["user_id"].astype(str)

  return dataframe  


In [251]:
df_books_copy = df_books.copy()
df_ratings_copy = df_ratings.copy()
df_ratings_copy['roberta_compound'] = - df_ratings_copy.roberta_neg + df_ratings_copy.roberta_neu + df_ratings_copy.roberta_pos

df_books_numeric_mod = preprocess_numeric_data(df_books_copy, 'df_books_copy')
df_books_processed = preprocess_text_data(df_books_numeric_mod, 'df_books_numeric_mod')
df_ratings_numeric_mod = preprocess_numeric_data(df_ratings_copy, 'df_users')
df_ratings_processed = preprocess_text_data(df_ratings_numeric_mod, 'df_users_numeric_mod')

In [252]:
df_books_processed.info(0)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115 entries, 0 to 114
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           115 non-null    object
 1   title             115 non-null    object
 2   description       115 non-null    object
 3   authors           115 non-null    object
 4   image             115 non-null    object
 5   previewLink       115 non-null    object
 6   publisher         115 non-null    object
 7   infoLink          115 non-null    object
 8   genre             115 non-null    object
 9   publication_year  115 non-null    int32 
dtypes: int32(1), object(9)
memory usage: 8.7+ KB


In [253]:
df_ratings_processed.info(0)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13842 entries, 0 to 13841
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Id                13842 non-null  int64  
 1   roberta_neg       13842 non-null  float32
 2   roberta_neu       13842 non-null  float32
 3   roberta_pos       13842 non-null  float32
 4   book_id           13842 non-null  object 
 5   title             13842 non-null  object 
 6   user_id           13842 non-null  object 
 7   profileName       13842 non-null  object 
 8   review_score      13842 non-null  float64
 9   review_text       13842 non-null  object 
 10  roberta_compound  13842 non-null  float32
dtypes: float32(4), float64(1), int64(1), object(5)
memory usage: 973.4+ KB


In [254]:
desired_columns = ['title', 'description', 'authors', 'image', 'previewLink',
       'publisher', 'infoLink', 'genre', 'publication_year']

df_ratings_books_processed = pd.merge(df_ratings_processed, df_books_processed[desired_columns], on='title', how='inner')
df_ratings_books_processed.head(1)

,Id,roberta_neg,roberta_neu,roberta_pos,book_id,title,user_id,profileName,review_score,review_text,roberta_compound,description,authors,image,previewLink,publisher,infoLink,genre,publication_year
0,1747,0.004265,0.053986,0.941749,B000N6DDJQ,The Scarlet Letter A Romance,AUM3YMZ0YRJE0,Robert J. Crawford,5.0,"WHen I finally started reading this classic, I...",0.99147,no description,other,https://www.google.com/url?sa=i&url=https%3A%2...,No Link,other,No Link,other,2010


**Save df_ratings_books_processed**

In [273]:
df_ratings_books_processed.to_parquet('C:/Users/adria/Documents/Projects/Books/Resources/DataFrames/SentimentAnalysis/200plusRatingsPerUser_60plusRatingsPerBook/df_ratings_books_processed.parquet', index=False)

In [274]:
df_books_processed.to_parquet('C:/Users/adria/Documents/Projects/Books/Resources/DataFrames/SentimentAnalysis/200plusRatingsPerUser_60plusRatingsPerBook/df_books_processed.parquet', index=False)

In [275]:
df_ratings_processed.to_parquet('C:/Users/adria/Documents/Projects/Books/Resources/DataFrames/SentimentAnalysis/200plusRatingsPerUser_60plusRatingsPerBook/df_ratings_processed.parquet', index=False)

### Info about df_ratings_books_processed

- number of entries 
- description
- features

# SVD Model

**With Review Score**

In [255]:
df_svd = df_ratings_books_processed[['user_id', 'book_id', 'review_score']]
reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(df_svd, reader)
trainset, testset = model_selection.train_test_split(data, test_size=0.30, random_state=42)
model = SVD()
model.fit(trainset)
predictions = model.test(testset)
accuracy.rmse(predictions)


RMSE: 0.5991


0.59907318486913

**With Sentiment Analysis**

In [304]:
df_svd_SA = df_ratings_books_processed[['user_id', 'book_id', 'roberta_compound']]
reader_SA = Reader(rating_scale=(-1, 1))
data_SA = Dataset.load_from_df(df_svd_SA, reader_SA)
trainset_SA, testset_SA = model_selection.train_test_split(data_SA, test_size=0.30, random_state=42)
model_SA = SVD()
model_SA.fit(trainset_SA)
predictions_SA = model_SA.test(testset_SA)
accuracy.rmse(predictions_SA)


RMSE: 0.3051


0.3050629046679383

The lower the RSME the better

In [305]:
with open('C:/Users/adria/Documents/Projects/Books/Resources/Models/BookRecommendation/model_svd.pkl', 'wb') as p:
    dump(model, p)

with open('C:/Users/adria/Documents/Projects/Books/Resources/Models/BookRecommendation/model_svd_SA.pkl', 'wb') as p:
    dump(model_SA, p)



In [258]:
def make_clickable(val):
    return '<a target="_blank" href="{}">Google Books</a>'.format(val, val)

def show_image(val):
    return '<a href="{}"><img src="{}" width=60></img></a>'.format(val, val)

In [301]:

def recommendation_svd(model_select, reader_select, metric_score):
  user_id = str(input('Enter user_id: '))
  book_id = set(df_ratings_books_processed[df_ratings_books_processed['user_id']==user_id]['book_id']) # has book ids that have been reviewed

  # Group df_ratings_books_processed by the book id. of all the books that haven't been reviewd by the user
    # Get the mean of review_score and mean of roberta_compound
  # Then merge the df_books_processed to extract 'previewLink', 'image' by 'book_id'

  user_books = df_ratings_books_processed[~df_ratings_books_processed['book_id'].isin(book_id)] # has all books that haven't been reviewd
  user_books['user_id'] = len(user_books)*[user_id]
  user_books.reset_index(drop=True, inplace=True)

  df_svd_predict = Dataset.load_from_df(user_books[['user_id', 'book_id', metric_score]], reader_select)
  NA, test = model_selection.train_test_split(df_svd_predict, test_size=1.0)
  predictions = model_select.test(test)
  predictions = [prediction.est for prediction in predictions]
  user_books['rating'] = predictions

  user_books_grouped = user_books.groupby("book_id").agg({
    "review_score": "mean",
    "roberta_compound": "mean",
    'rating':'mean'
  }).reset_index()

  user_books_merged = pd.merge(user_books_grouped, df_books_processed[['book_id', 'title', 'previewLink', 'image', 'genre', 'publisher', 'authors']].drop_duplicates(), how='inner', on='book_id')

  top_50_books_for_user_content = user_books_merged.sort_values(by=['rating'], ascending=False)[:50]
  top_50_books_for_user_content.to_csv('top_50_books_for_user_content.csv', index=False)
  book_title_liked_by_user = set(df_ratings_books_processed[df_ratings_books_processed['user_id']==user_id].sort_values(by=metric_score, ascending=False)['title'])
  print('Books highly rated by given user: \n')

  for count, books in tqdm(enumerate(list(book_title_liked_by_user)[:20])):
    print(count+1,'.  ',books)
  return top_50_books_for_user_content[['book_id', 'title', 'review_score', 'previewLink', 'image', 'rating']].head(10).style.format({'previewLink': make_clickable, 'image': show_image})


# // TODO: Fix titles, like filtering The hobbit
# // NOTE: AEGR7IBHUM3R7 User to use for book recommendations 

In [302]:
recommendation_svd(model_select=model, reader_select=reader, metric_score="review_score")

Books highly rated by given user: 



20it [00:00, ?it/s]

1 .   Fahrenheit 451
2 .   Fahrenheit 451 (Cascades S.)
3 .   Lord of the Flies
4 .   The Picture of Dorian Gray
5 .   The Scarlet Letter
6 .   The Hobbit
7 .   The Hobbit or There and Back Again
8 .   Animal Farm
9 .   The Picture of Dorian Gray (Classic Collection (Brilliance Audio))
10 .   Atlas Shrugged
11 .   To Kill a Mockingbird
12 .   Nineteen Eighty-four
13 .   Starship Troopers
14 .   To Kill A Mockingbird
15 .   Animal Farm (Signet Classics, CT304)
16 .   Alice's Adventures in Wonderland and Through the Looking Glass
17 .   Dracula (G. K. Hall (Large Print))
18 .   The Hobbit; Or, There and Back Again
19 .   The Old Man and the Sea
20 .   The War of the Worlds


,book_id,title,review_score,previewLink,image,rating
7,0446781819,Gone with the Wind,4.833333,Google Books,,4.901504
43,B000GLN7HQ,The Murder of Roger Ackroyd,4.750000,Google Books,,4.883156
49,B000IVDZR6,The Autobiography of Benjamin Franklin,4.500000,Google Books,,4.861367
69,B000PC4JLY,Hiroshima,5.000000,Google Books,,4.855801
44,B000GVE24S,Twenty Thousand Leagues Under the Sea,4.040000,Google Books,,4.851878
67,B000P1QRII,The Giver,4.687500,Google Books,,4.851263
13,0613175719,Ulysses,4.548387,Google Books,,4.847864
35,B0000CMQXI,A Portrait of the Artist as a Young Man,4.518519,Google Books,,4.847857
40,B0008CGH2K,An inquiry into the nature and causes of the wealth of nations,3.764706,Google Books,,4.847778
27,1593355548,Wuthering Heights,4.423077,Google Books,,4.845472


In [303]:
recommendation_svd(model_select=model_SA, reader_select=reader_SA, metric_score='roberta_compound')

Books highly rated by given user: 



20it [00:00, ?it/s]

1 .   Fahrenheit 451
2 .   Fahrenheit 451 (Cascades S.)
3 .   The Picture of Dorian Gray
4 .   Lord of the Flies
5 .   The Scarlet Letter
6 .   The Hobbit
7 .   The Hobbit or There and Back Again
8 .   Animal Farm
9 .   The Picture of Dorian Gray (Classic Collection (Brilliance Audio))
10 .   Atlas Shrugged
11 .   To Kill a Mockingbird
12 .   Nineteen Eighty-four
13 .   Starship Troopers
14 .   To Kill A Mockingbird
15 .   Animal Farm (Signet Classics, CT304)
16 .   Alice's Adventures in Wonderland and Through the Looking Glass
17 .   Dracula (G. K. Hall (Large Print))
18 .   The Hobbit; Or, There and Back Again
19 .   The Old Man and the Sea
20 .   The War of the Worlds


,book_id,title,review_score,previewLink,image,rating
64,B000NWUHR6,Catch-22,4.529412,Google Books,,0.671647
51,B000JJ9ISM,The Daughter of Time,4.800000,Google Books,,0.668624
26,158855032X,Hard Times,4.562500,Google Books,,0.667421
58,B000L6Q9ZS,The Stranger,4.272727,Google Books,,0.664526
73,B000PHIIM0,Leaves of Grass,4.916667,Google Books,,0.663267
50,B000J33RXK,Slaughterhouse-Five,4.451613,Google Books,,0.657082
63,B000NWO8CG,A Tree Grows in Brooklyn,4.862069,Google Books,,0.655754
60,B000N28H2I,The Two Towers,4.796875,Google Books,,0.651540
21,1556863047,The Sea Wolf,4.600000,Google Books,,0.651224
31,1901768619,Emma,4.603448,Google Books,,0.650798


In [262]:
# df_ratings_books_processed.head(20)

df_ratings_books_processed[df_ratings_books_processed['title'].str.contains('hobbit', case=False, na=False)].head(3)

,Id,roberta_neg,roberta_neu,roberta_pos,book_id,title,user_id,profileName,review_score,review_text,roberta_compound,description,authors,image,previewLink,publisher,infoLink,genre,publication_year
1845,86652,0.222972,0.677108,0.099919,B000NWU3I4,"The Hobbitt, or there and back again; illustra...",AEGR7IBHUM3R7,mp,4.0,"J.R.R. Tolkien's 1937 novel, ""The Hobbit: or T...",0.554055,bilbo baggins a respectable welltodo hobbit li...,J. R. R. Tolkien,http://books.google.com/books/content?id=Wbm1m...,http://books.google.nl/books?id=Wbm1mQEACAAJ&d...,houghton mifflin harcourt,http://books.google.nl/books?id=Wbm1mQEACAAJ&d...,Fiction,2013
1846,86697,0.014145,0.186257,0.799597,B000NWU3I4,"The Hobbitt, or there and back again; illustra...",A20EEWWSFMZ1PN,"bernie ""xyzzy""",5.0,"In a mythical or maybe just forgotten time, ma...",0.971709,bilbo baggins a respectable welltodo hobbit li...,J. R. R. Tolkien,http://books.google.com/books/content?id=Wbm1m...,http://books.google.nl/books?id=Wbm1mQEACAAJ&d...,houghton mifflin harcourt,http://books.google.nl/books?id=Wbm1mQEACAAJ&d...,Fiction,2013
1847,86783,0.006597,0.091661,0.901742,B000NWU3I4,"The Hobbitt, or there and back again; illustra...",A1F8GH7CR68P59,"A. G. Plumb ""Greg Plumb""",4.0,I refer to the Folio Edition version of this n...,0.986805,bilbo baggins a respectable welltodo hobbit li...,J. R. R. Tolkien,http://books.google.com/books/content?id=Wbm1m...,http://books.google.nl/books?id=Wbm1mQEACAAJ&d...,houghton mifflin harcourt,http://books.google.nl/books?id=Wbm1mQEACAAJ&d...,Fiction,2013


In [307]:
df_ratings_books_processed['genre'].nunique()

26

**Correlation Analysis**

In [ ]:
cormat = df_books_users[['publication_year', 'ratings_count', 'book_average_rating', 'num_pages', 'user_rating']].corr()
round(cormat,2)
plt.figure(figsize=(12, 8))
sns.heatmap(cormat, cmap='cividis', annot=True, linewidths=2)
plt.show()
     


**Word Cloud**

In [ ]:
from wordcloud import WordCloud

wc = WordCloud(width=500,height=500,min_font_size=15,background_color='white')
spam_wc = wc.generate(data[data['review/score'] > 3]['review/text'].str.cat(sep=" "))

plt.figure(figsize=(20,10))
plt.axis('off')
plt.imshow(spam_wc)

**Label Sentiment Analysis**

In [ ]:
data['Sentiment'] = data['compound'].apply(lambda x: 'positive' if x >= 0.05 else 'negative' if x < -0.05 else 'neutral')
data.head()

**PIE chart sentiment Analysis distibution**

In [ ]:
plt.rcParams['figure.figsize'] = (10, 10)
labels=['positive','negative','neutral ']
plt.pie(data['Sentiment'].value_counts(),explode =(0.1,0.1,0.1), labels=labels,autopct='%1.1f%%')
plt.title('Type ', fontsize = 15)
plt.axis('off')
plt.legend()
plt.show()

**Best 5 books accoding to sentiment analysis roberta compound**

In [ ]:
import streamlit as st
import pandas as pd
import plotly.express as px

# Sample DataFrame
data = {
    "Title": ["Book 1", "Book 2", "Book 3", "Book 4", "Book 5"],
    "roberta_compound": [0.8, 0.9, 0.7, 0.8, 0.9]
}

df = pd.DataFrame(data)

# Calculate the average roberta_compound score for each book
average_scores = df.groupby("Title")["roberta_compound"].mean().reset_index()

# Select the top 5 books
top_5_books = average_scores.nlargest(5, "roberta_compound")

# Plotly horizontal bar chart
fig = px.bar(
    top_5_books,
    x="roberta_compound",
    y="Title",
    orientation="h",
    labels={"roberta_compound": "Average Roberta Compound Score", "Title": "Book Title"},
    title="Top 5 Books by Average Roberta Compound Score",
)

# Display the chart in Streamlit
st.plotly_chart(fig, use_container_width=True)


# Step 4. SVD Model with Sentiment Analysis and NNN

**Create Pivot Table**

In [ ]:
combined = reviews_filtered.drop_duplicates(['User-ID', 'Book-ID'])
user_book_matrix = combined.pivot(index='User-ID', columns='Book-ID', values='ReviewRating')
user_book_matrix.fillna(0, inplace=True)

user_book_matrix.head(10)

**Create Embeddings**

In [ ]:
from scipy.sparse.linalg import svds


def build_SVD_embeddings(user_book_matrix, embedding_length):
    #Builds SVD embeddings to be used later on
    U, Sigma, VT = svds(user_book_matrix, k = embedding_length)
    user_embed_df = pd.DataFrame(U, index = user_book_matrix.index)
    VT_T = np.transpose(VT)
    book_embed_df = pd.DataFrame(VT_T, index = user_book_matrix.columns)
    
    return user_embed_df, book_embed_df

In [ ]:
embedding_length = 64
user_embed_df, book_embed_df = build_SVD_embeddings(user_book_matrix, embedding_length)
